<a href="https://colab.research.google.com/github/nicosesma/otc_strategies/blob/main/BTC_DCA_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class BTC_DCA_Strategy:
    """
    Implements a Dollar-Cost Averaging (DCA) trading strategy with a set number of ladders.
    This class can be used to generate trading signals within a backtesting environment.
    """

    def __init__(self,
                 base_order_size: float,
                 dca_ladder_steps: list,
                 take_profit_percent: float):
        """
        Initializes the DCA strategy with core parameters.

        Args:
            base_order_size (float): The size of the initial base order in dollars.
            dca_ladder_steps (list): A list of price drops (as a percentage) to trigger
                                     each subsequent DCA order.
            take_profit_percent (float): The percentage gain on the average order price
                                         at which the trade will be closed.
        """
        self.base_order_size = base_order_size
        self.dca_ladder_steps = [step / 100.0 for step in dca_ladder_steps]
        self.take_profit_percent = take_profit_percent / 100.0
        self.dca_ladder_multipliers = [1.5, 2.0, 3.0, 4.0, 5.0] # You can customize these multipliers

    def run_backtest(self, dataframe: pd.DataFrame):
        """
        Simulates the DCA trading strategy on the provided historical data.

        Args:
            dataframe (pd.DataFrame): A pandas DataFrame with a 'close' price column.

        Returns:
            dict: A dictionary containing the trade history and final PnL.
        """
        trade_open = False
        trade_info = {}
        trade_history = []

        for i, row in dataframe.iterrows():
            current_price = row['close']

            # --- Entry Logic (Place a Base Order) ---
            # For this simulation, we'll open a trade on the very first data point.
            if not trade_open and i == dataframe.index[0]:
                base_order_price = current_price
                base_order_size_btc = self.base_order_size / base_order_price

                trade_info = {
                    'entry_price': base_order_price,
                    'total_btc_volume': base_order_size_btc,
                    'total_cost': self.base_order_size,
                    'avg_price': base_order_price,
                    'orders_placed': 1,
                    'is_dca_ladders_active': True
                }
                trade_history.append({
                    'Order': 'Base Order',
                    'Price': base_order_price,
                    'Size (BTC)': base_order_size_btc,
                    'Avg Price': base_order_price
                })
                trade_open = True

            # --- DCA Ladder Logic (Place Averaging Orders) ---
            if trade_open and trade_info['is_dca_ladders_active']:
                # Determine how many DCA orders we have left to place
                ladders_to_go = len(self.dca_ladder_steps) - (trade_info['orders_placed'] - 1)

                # Check if the price has dropped enough to trigger the next DCA order
                if ladders_to_go > 0:
                    dca_step_index = trade_info['orders_placed'] - 1
                    price_deviation = (trade_info['avg_price'] - current_price) / trade_info['avg_price']

                    if price_deviation >= self.dca_ladder_steps[dca_step_index]:
                        # A new DCA order is triggered
                        order_multiplier = self.dca_ladder_multipliers[dca_step_index]
                        dca_order_size_usd = self.base_order_size * order_multiplier
                        dca_order_size_btc = dca_order_size_usd / current_price

                        # Update trade state
                        new_total_cost = trade_info['total_cost'] + dca_order_size_usd
                        new_total_btc = trade_info['total_btc_volume'] + dca_order_size_btc

                        trade_info['total_cost'] = new_total_cost
                        trade_info['total_btc_volume'] = new_total_btc
                        trade_info['avg_price'] = new_total_cost / new_total_btc
                        trade_info['orders_placed'] += 1

                        trade_history.append({
                            'Order': f'DCA Order {trade_info["orders_placed"]-1}',
                            'Price': current_price,
                            'Size (BTC)': dca_order_size_btc,
                            'Avg Price': trade_info['avg_price']
                        })
                else:
                    trade_info['is_dca_ladders_active'] = False

            # --- Take Profit Logic ---
            if trade_open:
                take_profit_price = trade_info['avg_price'] * (1 + self.take_profit_percent)

                if current_price >= take_profit_price:
                    pnl = (current_price - trade_info['avg_price']) * trade_info['total_btc_volume']
                    final_roi = (pnl / trade_info['total_cost']) * 100
                    trade_history.append({
                        'Order': 'Take Profit',
                        'Price': current_price,
                        'PnL ($)': pnl,
                        'ROI (%)': final_roi
                    })
                    trade_open = False # Close the trade
                    break # Exit the loop after closing trade

        # If the loop finishes and the trade is still open, calculate current PnL
        if trade_open:
            pnl = (current_price - trade_info['avg_price']) * trade_info['total_btc_volume']
            final_roi = (pnl / trade_info['total_cost']) * 100
            trade_history.append({
                'Order': 'Final Position (Unclosed)',
                'Price': current_price,
                'PnL ($)': pnl,
                'ROI (%)': final_roi
            })

        return trade_history

def generate_mock_btc_data(start_date: datetime, end_date: datetime):
    """
    Generates a Pandas DataFrame with simulated BTC price data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    n_points = len(date_range)

    np.random.seed(42)  # For reproducibility
    initial_price = 65000 # Starting BTC price
    price_changes = np.random.normal(0, 1500, n_points) / initial_price
    price_series = initial_price * (1 + price_changes).cumprod()

    # Simulating a large price drop to trigger DCA ladders
    price_series[20:30] *= np.linspace(0.98, 0.90, 10)

    df = pd.DataFrame(price_series, index=date_range, columns=['close'])
    return df

if __name__ == '__main__':
    # --- 1. Define the DCA Parameters ---
    dca_params = {
        'base_order_size': 100.0,
        'dca_ladder_steps': [2.0, 4.0, 6.0, 8.0, 10.0], # 5 ladders
        'take_profit_percent': 5.0
    }

    # --- 2. Generate Mock Data ---
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2025, 1, 1)
    mock_data = generate_mock_btc_data(start_date, end_date)

    # --- 3. Instantiate the Strategy ---
    strategy = BTC_DCA_Strategy(**dca_params)

    # --- 4. Run the Backtest and Get Results ---
    print("Running a simulated backtest...")
    trade_results = strategy.run_backtest(mock_data)

    # --- 5. Print the Final Report ---
    print("\n--- BTC DCA Algorithm Backtest Report ---")

    if not trade_results:
        print("No trades were opened or closed during the simulation.")
    else:
        print("Order History:")
        for order in trade_results:
            print(f"  - {order['Order']:<25}: Price: ${order['Price']:.2f}, Size: {order.get('Size (BTC)', 0):.6f} BTC")
            if 'Avg Price' in order:
                print(f"    Average Order Price: ${order['Avg Price']:.2f}")
            if 'PnL ($)' in order:
                print(f"    PnL: ${order['PnL ($)']:.2f}, ROI: {order['ROI (%)']:.2f}%")

        last_order = trade_results[-1]
        print("\n--- Final Summary ---")
        if last_order['Order'] == 'Take Profit':
            print(f"Trade successfully closed for a profit.")
            print(f"Final Profit/Loss: ${last_order['PnL ($)']:.2f}")
            print(f"Final ROI: {last_order['ROI (%)']:.2f}%")
        else:
            print("Trade is still open at the end of the simulation.")
            print(f"Current Profit/Loss: ${last_order['PnL ($)']:.2f}")
            print(f"Current ROI: {last_order['ROI (%)']:.2f}%")